In [2]:
import os, random, time
from pathlib import Path

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from torch.cuda.amp import autocast, GradScaler

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
from PIL import Image

# Reproducibility
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
DATA_ROOT = Path("D:\ie4483 dataset\datasets")  
TRAIN_DIR = DATA_ROOT / "train"
VAL_DIR   = DATA_ROOT / "val"
TEST_DIR  = DATA_ROOT / "test"

CFG = {
    "img_h": 150,
    "img_w": 150,
    "batch_size": 32,
    "epochs": 30,
    "lr": 3e-4,
    "weight_decay": 1e-4,
    "num_workers": 4,
    "train_limit_per_class": None,  
    "val_limit_per_class":   None,  
    "save_dir": "checkpoints",
    "model_name": "alexnet_150.pt",
}
os.makedirs(CFG["save_dir"], exist_ok=True)
CFG


<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\chuan\AppData\Local\Temp\ipykernel_34192\503007766.py:1: SyntaxWarning: invalid escape sequence '\i'
  DATA_ROOT = Path("D:\ie4483 dataset\datasets")  # <-- CHANGE ME


{'img_h': 150,
 'img_w': 150,
 'batch_size': 32,
 'epochs': 30,
 'lr': 0.0003,
 'weight_decay': 0.0001,
 'num_workers': 4,
 'train_limit_per_class': None,
 'val_limit_per_class': None,
 'save_dir': 'checkpoints',
 'model_name': 'alexnet_150.pt'}

In [4]:
img_size = (CFG["img_h"], CFG["img_w"])

train_tfms = transforms.Compose([
    transforms.Resize(img_size, antialias=True),
    transforms.RandomAffine(
        degrees=40,
        translate=(0.2, 0.2),
        scale=(0.8, 1.2),
        shear=0.2
    ),
    transforms.RandomHorizontalFlip(p=0.5),  # matches Keras horizontal_flip=True
    transforms.ToTensor(),  # rescales to [0,1]
])

val_tfms = transforms.Compose([
    transforms.Resize(img_size, antialias=True),
    transforms.ToTensor(),  # rescales to [0,1], no aug
])


In [5]:
train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_tfms)
val_ds   = datasets.ImageFolder(VAL_DIR,   transform=val_tfms)
print(train_ds.classes, train_ds.class_to_idx)  # expect ['cat','dog'] and {'cat':0,'dog':1}


['cat', 'dog'] {'cat': 0, 'dog': 1}


In [6]:
def cap_per_class_indices(dataset, limit_per_class=None):
    if limit_per_class is None:
        return np.arange(len(dataset))
    targets = [dataset.samples[i][1] for i in range(len(dataset))]
    idx_by_class = {c: [] for c in set(targets)}
    for i, t in enumerate(targets):
        if len(idx_by_class[t]) < limit_per_class:
            idx_by_class[t].append(i)
    capped = np.concatenate([idx_by_class[c] for c in sorted(idx_by_class.keys())])
    return capped

CFG["train_limit_per_class"] = 5000
CFG["val_limit_per_class"]   = 1250


train_indices = cap_per_class_indices(train_ds, CFG["train_limit_per_class"])
val_indices   = cap_per_class_indices(val_ds,   CFG["val_limit_per_class"])

train_ds_cap = Subset(train_ds, train_indices)
val_ds_cap   = Subset(val_ds,   val_indices)
len(train_ds_cap), len(val_ds_cap)


(10000, 2500)

In [7]:
train_loader = DataLoader(
    train_ds_cap, batch_size=CFG["batch_size"], shuffle=True,
    num_workers=CFG["num_workers"], pin_memory=True
)
val_loader = DataLoader(
    val_ds_cap, batch_size=CFG["batch_size"], shuffle=False,
    num_workers=CFG["num_workers"], pin_memory=True
)


In [8]:
model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
in_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG["epochs"])
scaler = GradScaler(enabled=(device.type=="cuda"))

sum(p.numel() for p in model.parameters())/1e6


C:\Users\chuan\AppData\Local\Temp\ipykernel_34192\1932791875.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(device.type=="cuda"))


57.012034

In [21]:
def run_epoch(loader, model, optimizer=None, scaler=None):
    is_train = optimizer is not None
    if is_train: model.train()
    else: model.eval()

    total_loss, all_preds, all_tgts = 0.0, [], []
    for imgs, tgts in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        tgts = tgts.to(device, non_blocking=True)

        with autocast(enabled=(device.type=="cuda")):
            logits = model(imgs)
            loss = criterion(logits, tgts)

        if is_train:
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        total_loss += loss.item() * imgs.size(0)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_tgts.append(tgts.detach().cpu().numpy())

    avg_loss = total_loss / len(loader.dataset)
    all_preds = np.concatenate(all_preds); all_tgts = np.concatenate(all_tgts)
    acc = accuracy_score(all_tgts, all_preds)
    return avg_loss, acc, all_preds, all_tgts

best_acc, best_path = 0.0, os.path.join(CFG["save_dir"], CFG["model_name"])
history = []

for epoch in range(1, CFG["epochs"]+1):
    t0 = time.time()
    train_loss, train_acc, _, _ = run_epoch(train_loader, model, optimizer, scaler)
    val_loss, val_acc, vp, vt = run_epoch(val_loader, model, None, None)
    scheduler.step()

    history.append({
        "epoch": epoch,
        "train_loss": train_loss, "train_acc": train_acc,
        "val_loss": val_loss, "val_acc": val_acc,
        "lr": scheduler.get_last_lr()[0]
    })
    print(f"Epoch {epoch:02d} | "
          f"Train {train_loss:.4f}/{train_acc:.4f} | "
          f"Val {val_loss:.4f}/{val_acc:.4f} | "
          f"lr {scheduler.get_last_lr()[0]:.2e} | "
          f"{time.time()-t0:.1f}s")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            "model_state": model.state_dict(),
            "class_to_idx": train_ds.class_to_idx,
            "cfg": CFG
        }, best_path)
        print(f"  ✔ Saved new best to {best_path} (val_acc={best_acc:.4f})")

pd.DataFrame(history)


  0%|          | 0/313 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
  0%|          | 0/79 [00:00<?, ?it/s]           C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):


Epoch 01 | Train 0.4737/0.7742 | Val 0.2873/0.8652 | lr 2.99e-04 | 134.2s
  ✔ Saved new best to checkpoints\alexnet_150.pt (val_acc=0.8652)


  0%|          | 0/313 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
  0%|          | 0/79 [00:00<?, ?it/s]           C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp

Epoch 02 | Train 0.3269/0.8591 | Val 0.2542/0.9028 | lr 2.97e-04 | 134.0s
  ✔ Saved new best to checkpoints\alexnet_150.pt (val_acc=0.9028)


  0%|          | 0/313 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
  0%|          | 0/79 [00:00<?, ?it/s]           C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp

...

  0%|          | 0/313 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
  0%|          | 0/79 [00:00<?, ?it/s]           C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp

Epoch 29 | Train 0.0570/0.9779 | Val 0.1816/0.9556 | lr 8.22e-07 | 137.3s


  0%|          | 0/313 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
  0%|          | 0/79 [00:00<?, ?it/s]           C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp

Epoch 30 | Train 0.0556/0.9779 | Val 0.1808/0.9560 | lr 0.00e+00 | 139.0s


,epoch,train_loss,train_acc,val_loss,val_acc,lr
0,1,0.473652,0.7742,0.287269,0.8652,2.991783e-04
1,2,0.326903,0.8591,0.254204,0.9028,2.967221e-04
2,3,0.304356,0.8670,0.228077,0.9088,2.926585e-04
3,4,0.275173,0.8846,0.289079,0.9020,2.870318e-04
4,5,0.248456,0.8901,0.233533,0.9184,2.799038e-04
5,6,0.240493,0.9012,0.199413,0.9164,2.713525e-04
6,7,0.226812,0.9079,0.176040,0.9300,2.614717e-04
7,8,0.209659,0.9142,0.161583,0.9404,2.503696e-04
8,9,0.204092,0.9192,0.157629,0.9336,2.381678e-04
9,10,0.194782,0.9227,0.146496,0.9412,2.250000e-04


In [22]:
ckpt = torch.load(best_path, map_location=device)
model.load_state_dict(ckpt["model_state"])
model.eval()

_, _, preds, tgts = run_epoch(val_loader, model, None, None)
print("Val accuracy:", accuracy_score(tgts, preds))
print(classification_report(tgts, preds, target_names=train_ds.classes))

cm = confusion_matrix(tgts, preds)
cm


  0%|          | 0/79 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\chuan\AppData\Local\Temp\ipykernel_424\2411271896.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
                                               

Val accuracy: 0.956
              precision    recall  f1-score   support

         cat       0.96      0.96      0.96      1250
         dog       0.96      0.96      0.96      1250

    accuracy                           0.96      2500
   macro avg       0.96      0.96      0.96      2500
weighted avg       0.96      0.96      0.96      2500



array([[1194,   56],
       [  54, 1196]])

In [ ]:
# rebuild the test DataLoader with 0 workers to avoid Windows multiprocessing issues
test_ds = TestImageFolder(TEST_DIR, val_tfms)
print("Test images found:", len(test_ds))

test_dl = DataLoader(
    test_ds,
    batch_size=CFG["batch_size"],
    shuffle=False,
    num_workers=0,               # <— key change
    pin_memory=False,            
    persistent_workers=False     # ensure workers aren't kept alive
)

# re-run inference
all_ids, all_preds = [], []
model.eval()
with torch.no_grad():
    for imgs, paths in tqdm(test_dl):
        imgs = imgs.to(device, non_blocking=True)
        with autocast(enabled=(device.type=="cuda")):
            logits = model(imgs)
        pred_idx = torch.argmax(logits, dim=1).detach().cpu().numpy()
        all_preds.extend(pred_idx.tolist())
        all_ids.extend([Path(p).stem for p in paths])

sub_df = pd.DataFrame({"ID": all_ids, "label": all_preds})
sub_df.to_csv("submission.csv", index=False)
print("Wrote submission.csv with", len(sub_df), "rows")


Test images found: 500


  0%|          | 0/16 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Temp\ipykernel_424\375779877.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=="cuda")):
100%|██████████| 16/16 [00:09<00:00,  1.70it/s]

Wrote submission.csv with 500 rows


In [25]:
import os
print("CWD:", os.getcwd())
print("Exists?", os.path.exists("submission.csv"))


CWD: c:\Users\chuan\AppData\Local\Programs\Microsoft VS Code
Exists? True


In [26]:
out_csv = DATA_ROOT / "submission.csv"
sub_df.to_csv(str(out_csv), index=False)  # str() for Windows paths
print("Saved to:", out_csv.resolve())


Saved to: D:\ie4483 dataset\datasets\submission.csv


In [ ]:
# ---- CIFAR-10 configuration ----
CIFAR = {
    "img_size": 224,        # upscale 32x32 -> 224x224 to leverage ImageNet-pretrained AlexNet
    "batch_size": 128,
    "epochs": 30,           # typical for CIFAR-10
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "num_workers": 2,       # set to 0 on Windows if workers crash
    "save_dir": "checkpoints",
    "model_name": "alexnet_cifar10.pt",
    "use_mixup": False,     # optional regularization
    "mixup_alpha": 0.2
}
import os
os.makedirs(CIFAR["save_dir"], exist_ok=True)
print(CIFAR)


{'img_size': 224, 'batch_size': 128, 'epochs': 30, 'lr': 0.001, 'weight_decay': 0.0001, 'num_workers': 2, 'save_dir': 'checkpoints', 'model_name': 'alexnet_cifar10.pt', 'use_mixup': False, 'mixup_alpha': 0.2}


In [30]:
from torchvision import datasets, transforms

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms_c10 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Resize((CIFAR["img_size"], CIFAR["img_size"]), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

test_tfms_c10 = transforms.Compose([
    transforms.Resize((CIFAR["img_size"], CIFAR["img_size"]), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Download & build datasets
cifar10_train = datasets.CIFAR10(root="./data", train=True,  download=True, transform=train_tfms_c10)
cifar10_test  = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tfms_c10)

cifar10_classes = cifar10_train.classes
print("CIFAR-10 classes:", cifar10_classes)


CIFAR-10 classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [31]:
# === Cell C (REPLACEMENT): DataLoaders for CIFAR-10 with 10k train / 2k test ===
from torch.utils.data import DataLoader, Subset
import numpy as np

# Reproducible stratified sampling: 1,000 per class for train, 200 per class for test
train_per_class = 2000
test_per_class  = 400
num_classes = 10
rng = np.random.default_rng(42)  # keep consistent with your notebook seed

def stratified_indices(dataset, per_class):
    # torchvision CIFAR10 stores numeric labels in dataset.targets (list[int])
    targets = np.array(dataset.targets)
    indices = []
    for c in range(num_classes):
        cls_idx = np.where(targets == c)[0]
        pick = rng.choice(cls_idx, size=per_class, replace=False)
        indices.append(pick)
    return np.concatenate(indices)

train_idx_small = stratified_indices(cifar10_train, train_per_class)  # 10 * 1000 = 20,000
test_idx_small  = stratified_indices(cifar10_test,  test_per_class)   # 10 * 200  = 4,000

cifar10_train_small = Subset(cifar10_train, train_idx_small)
cifar10_test_small  = Subset(cifar10_test,  test_idx_small)

print("Train (small) size:", len(cifar10_train_small))  # -> 10000
print("Test  (small) size:", len(cifar10_test_small))   # -> 2000

# DataLoaders (set num_workers=0 on Windows if you hit worker crashes)
train_loader_c10 = DataLoader(
    cifar10_train_small,
    batch_size=CIFAR["batch_size"],
    shuffle=True,                       # stratified subset already; still shuffle batches
    num_workers=CIFAR["num_workers"],
    pin_memory=True
)

test_loader_c10 = DataLoader(
    cifar10_test_small,
    batch_size=CIFAR["batch_size"],
    shuffle=False,
    num_workers=CIFAR["num_workers"],
    pin_memory=True
)


Train (small) size: 20000
Test  (small) size: 4000


In [32]:
import torch
from torch import nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alexnet_c10 = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
# replace final Linear (4096 -> 1000) with (4096 -> 10)
in_features = alexnet_c10.classifier[-1].in_features
alexnet_c10.classifier[-1] = nn.Linear(in_features, 10)
alexnet_c10 = alexnet_c10.to(device)

criterion_c10 = nn.CrossEntropyLoss()
optimizer_c10 = torch.optim.AdamW(alexnet_c10.parameters(), lr=CIFAR["lr"], weight_decay=CIFAR["weight_decay"])
scheduler_c10 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_c10, T_max=CIFAR["epochs"])
scaler_c10 = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

sum(p.numel() for p in alexnet_c10.parameters())/1e6


C:\Users\chuan\AppData\Local\Temp\ipykernel_34192\1648613737.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_c10 = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))


57.04481

In [33]:
import numpy as np
import torch.nn.functional as F

def mixup_data(x, y, alpha=0.2):
    if alpha <= 0: return x, y, torch.ones(len(x), device=x.device)
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0), device=x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, (y_a, y_b), lam

def mixup_criterion(criterion, preds, target_pair, lam):
    y_a, y_b = target_pair
    return lam * criterion(preds, y_a) + (1 - lam) * criterion(preds, y_b)


In [34]:
from tqdm import tqdm

def run_epoch_c10(loader, model, optimizer=None, scaler=None, use_mixup=False, alpha=0.2):
    is_train = optimizer is not None
    model.train(is_train)
    total, correct, total_loss = 0, 0, 0.0

    for imgs, targets in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            if is_train and use_mixup:
                xm, tpair, lam = mixup_data(imgs, targets, alpha=alpha)
                logits = model(xm)
                loss = mixup_criterion(criterion_c10, logits, tpair, lam)
                preds = logits.argmax(1)
                hard_targets = targets  # for accuracy proxy
            else:
                logits = model(imgs)
                loss = criterion_c10(logits, targets)
                preds = logits.argmax(1)
                hard_targets = targets

        if is_train:
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        total_loss += loss.item() * imgs.size(0)
        correct += (preds == hard_targets).sum().item()
        total += imgs.size(0)

    return total_loss/total, correct/total

@torch.no_grad()
def evaluate_c10(loader, model):
    model.eval()
    total, correct, total_loss = 0, 0, 0.0
    all_preds, all_tgts = [], []
    for imgs, targets in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(imgs)
            loss = criterion_c10(logits, targets)
            preds = logits.argmax(1)
        total_loss += loss.item() * imgs.size(0)
        correct += (preds == targets).sum().item()
        total += imgs.size(0)
        all_preds.append(preds.cpu().numpy())
        all_tgts.append(targets.cpu().numpy())

    import numpy as np
    all_preds = np.concatenate(all_preds)
    all_tgts = np.concatenate(all_tgts)
    return total_loss/total, correct/total, all_preds, all_tgts


In [35]:
best_acc_c10 = 0.0
best_path_c10 = os.path.join(CIFAR["save_dir"], CIFAR["model_name"])
history_c10 = []

for epoch in range(1, CIFAR["epochs"]+1):
    t0 = time.time()
    tr_loss, tr_acc = run_epoch_c10(
        train_loader_c10,
        alexnet_c10,
        optimizer_c10,
        scaler_c10,
        use_mixup=CIFAR["use_mixup"],
        alpha=CIFAR["mixup_alpha"]
    )
    te_loss, te_acc, _, _ = evaluate_c10(test_loader_c10, alexnet_c10)
    scheduler_c10.step()

    history_c10.append({
        "epoch": epoch,
        "train_loss": tr_loss, "train_acc": tr_acc,
        "test_loss": te_loss, "test_acc": te_acc,
        "lr": scheduler_c10.get_last_lr()[0]
    })
    print(f"Epoch {epoch:02d} | "
          f"Train {tr_loss:.4f}/{tr_acc:.4f} | "
          f"Test {te_loss:.4f}/{te_acc:.4f} | "
          f"lr {scheduler_c10.get_last_lr()[0]:.2e} | {time.time()-t0:.1f}s")

    if te_acc > best_acc_c10:
        best_acc_c10 = te_acc
        torch.save({
            "model_state": alexnet_c10.state_dict(),
            "classes": cifar10_classes,
            "cfg": CIFAR
        }, best_path_c10)
        print(f"  ✔ Saved new best to {best_path_c10} (test_acc={best_acc_c10:.4f})")


  0%|          | 0/157 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Temp\ipykernel_34192\2009227763.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
  0%|          | 0/32 [00:00<?, ?it/s]           C:\Users\chuan\AppData\Local\Temp\ipykernel_34192\2009227763.py:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):


Epoch 01 | Train 2.1363/0.1981 | Test 1.9411/0.2950 | lr 9.97e-04 | 258.1s
  ✔ Saved new best to checkpoints\alexnet_cifar10.pt (test_acc=0.2950)


Epoch 02 | Train 1.7789/0.3276 | Test 1.6069/0.4045 | lr 9.89e-04 | 260.8s
  ✔ Saved new best to checkpoints\alexnet_cifar10.pt (test_acc=0.4045)


Epoch 03 | Train 1.5637/0.4125 | Test 1.3541/0.4973 | lr 9.76e-04 | 254.3s
  ✔ Saved new best to checkpoints\alexnet_cifar10.pt (test_acc=0.4973)


Epoch 04 | Train 1.3984/0.4780 | Test 1.3561/0.4682 | lr 9.57e-04 | 255.9s


...

Epoch 26 | Train 0.2726/0.9071 | Test 0.5924/0.8287 | lr 4.32e-05 | 263.5s
  ✔ Saved new best to checkpoints\alexnet_cifar10.pt (test_acc=0.8287)


Epoch 27 | Train 0.2621/0.9081 | Test 0.6044/0.8247 | lr 2.45e-05 | 264.1s


Epoch 28 | Train 0.2552/0.9114 | Test 0.6002/0.8285 | lr 1.09e-05 | 264.0s


Epoch 29 | Train 0.2529/0.9135 | Test 0.5946/0.8283 | lr 2.74e-06 | 263.7s


Epoch 30 | Train 0.2509/0.9115 | Test 0.5938/0.8270 | lr 0.00e+00 | 263.5s


In [36]:
import torch
from sklearn.metrics import classification_report, confusion_matrix

ckpt = torch.load(best_path_c10, map_location=device)
alexnet_c10.load_state_dict(ckpt["model_state"])

test_loss, test_acc, preds, tgts = evaluate_c10(test_loader_c10, alexnet_c10)
print(f"\nBest AlexNet CIFAR-10 test accuracy: {test_acc:.4f}\n")
print(classification_report(tgts, preds, target_names=cifar10_classes, digits=4))

cm = confusion_matrix(tgts, preds)
cm


  0%|          | 0/32 [00:00<?, ?it/s]C:\Users\chuan\AppData\Local\Temp\ipykernel_34192\2009227763.py:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
                                               


Best AlexNet CIFAR-10 test accuracy: 0.8287

              precision    recall  f1-score   support

    airplane     0.8420    0.8125    0.8270       400
  automobile     0.9295    0.9225    0.9260       400
        bird     0.7291    0.7200    0.7245       400
         cat     0.7216    0.7000    0.7107       400
        deer     0.7694    0.8425    0.8043       400
         dog     0.7881    0.7625    0.7751       400
        frog     0.8453    0.9150    0.8788       400
       horse     0.8770    0.8200    0.8475       400
        ship     0.9015    0.9150    0.9082       400
       truck     0.8864    0.8775    0.8819       400

    accuracy                         0.8287      4000
   macro avg     0.8290    0.8287    0.8284      4000
weighted avg     0.8290    0.8287    0.8284      4000



array([[325,   7,  22,   5,   4,   0,   0,   8,  24,   5],
       [  2, 369,   0,   2,   0,   0,   2,   0,   5,  20],
       [ 16,   1, 288,  13,  30,  15,  25,   8,   1,   3],
       [  3,   0,  17, 280,  21,  46,  19,   4,   5,   5],
       [  3,   0,  22,  13, 337,   4,  13,   8,   0,   0],
       [  1,   0,  16,  51,  12, 305,   2,  11,   0,   2],
       [  2,   0,   9,   8,  11,   0, 366,   1,   1,   2],
       [  4,   0,  14,   9,  22,  17,   2, 328,   0,   4],
       [ 17,   2,   5,   2,   1,   0,   2,   1, 366,   4],
       [ 13,  18,   2,   5,   0,   0,   2,   5,   4, 351]])